# New Class Structure for Zi & Qi Cayley-Dickson Construction

In [11]:
from abc import ABC  #, abstractmethod
from fractions import Fraction

In [15]:
class CayleyDicksonBase(ABC):

    def __init__(self, real=None, imag=None):
        self._re = real
        self._im = imag

    @property
    def real(self):
        return self._re

    @property
    def imag(self):
        return self._im

    def __repr__(self):
        return f"{self.__class__.__name__}({self.real}, {self.imag})"

    def __hash__(self):
        return hash((self.real, self.imag, type(self)))

    def __len__(self):
        return 2

    def __getitem__(self, index):
        if isinstance(index, int):
            if index == 0:
                return self.real
            elif index == 1:
                return self.imag
            else:
                raise IndexError(f"Index {index} out of range")
        else:
            raise TypeError(f"Index {index} must be a non-negative integer")

    def __iter__(self):
        return iter((self.real, self.imag))

    def __eq__(self, other):
        if type(self) == type(other):
            return self.real == other.real and self.imag == other.imag
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)


class Zi(CayleyDicksonBase):

    def __init__(self, real=None, imag=None):
        
        if isinstance(real, (float, int)):
            if imag is None:
                super().__init__(round(real), 0)
            elif isinstance(imag, (float, int)):
                super().__init__(round(real), round(imag))
            else:
                raise Exception(f"Inputs incompatible: {real} and {imag}")
        
        elif isinstance(real, complex):
            if imag is None:
                super().__init__(round(real.real), round(real.imag))
            elif isinstance(imag, (complex, Zi)):
                super().__init__(Zi(real), Zi(imag))
            else:
                raise Exception(f"Inputs incompatible: {real} and {imag}")
                
        elif isinstance(real, Zi):
            if imag is None:
                super().__init__(real.real, real.imag)
            elif isinstance(imag, (complex, Zi)):
                super().__init__(Zi(real), Zi(imag))
            else:
                raise Exception(f"Inputs incompatible: {real} and {imag}")

        elif real is None:
            if imag is None:
                super().__init__(0, 0)
            else:
                raise Exception(f"If re is None, then im must be None. But im = {imag}")
        
        else:
            raise Exception("We should never get to this point in the code")

    def __eq__(self, other):
        if isinstance(other, Zi):
            return self.real == other.real and self.imag == other.imag
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    @staticmethod
    def zero():
        return Zi(0, 0)
    
    @staticmethod
    def eye():
        return Zi(0, 1)
    
    @staticmethod
    def two():
        return Zi(1, 1)

    def __add__(self, other):
        if isinstance(other, Zi):
            return Zi(self.real + other.real, self.imag + other.imag)
        else:
            raise TypeError("Can only add Zi objects")

    def __sub__(self, other):
        if isinstance(other, Zi):
            return Zi(self.real - other.real, self.imag - other.imag)
        else:
            raise TypeError("Can only subtract Zi objects")


class Qi(CayleyDicksonBase):

    __MAX_DENOMINATOR = 1_000_000

    def __init__(self, real=None, imag=None):

        # CLAUSE AA --------------------------------------------------------
        # real is a str, float, int, or Fraction; and
        # imag is a str, float, int, Fraction, or None

        if isinstance(real, (str, float, int, Fraction)):
            if isinstance(real, Fraction):
                re = real
            else:
                re = Fraction(real)
            if imag is None:
                im = Fraction(0)
            elif isinstance(imag, (str, float, int, Fraction)):
                if isinstance(imag, Fraction):
                    im = imag
                else:
                    im = Fraction(imag)
            else:
                raise Exception(f"Inputs incompatible: {real} and {imag}")
            super().__init__(re, im)

        # CLAUSE BA --------------------------------------------------------
        # real is a complex; and
        # imag is complex, a Qi, or None

        elif isinstance(real, complex):
            if imag is None:
                re = Fraction(real.real)
                im = Fraction(real.imag)
            elif isinstance(imag, (Qi, complex)):
                re = Qi(real.real, real.imag)
                im = Qi(imag.real, imag.imag)
            else:
                raise Exception(f"Inputs incompatible: {real} and {imag}")
            super().__init__(re, im)

        # CLAUSE CA --------------------------------------------------------
        # real is a Qi, and imag is complex, a Qi, or None

        elif isinstance(real, Qi):
            if imag is None:
                re = real.real
                im = real.imag
            elif isinstance(imag, (complex, Qi)):
                re = Qi(real)
                im = Qi(imag)
            else:
                raise Exception(f"Inputs incompatible: {real} and {imag}")
            super().__init__(re, im)

        # CLAUSE DA --------------------------------------------------------
        # real is a Zi, and imag is a Zi, or None

        elif isinstance(real, Zi):
            if imag is None:
                if real.is_complex():
                    re = Fraction(real.real)
                    im = Fraction(real.imag)
                else:
                    re = Qi(real.real)
                    im = Qi(real.imag)
            elif isinstance(imag, Zi) and imag.order() == real.order():
                    re = Qi(real)
                    im = Qi(imag)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")
            super().__init__(re, im)

        # CLAUSE EA --------------------------------------------------------
        # real is a list or tuple of numbers with length equal to a
        # power of 2, and imag is None, or it is a tuple or list
        # similar to the one input for real.
        
        elif isinstance(real, (tuple, list)):
            z = Zi.from_array(real)
            if imag is None:
                re = z.real
                im = z.imag
            elif isinstance(imag, (tuple, list)) and len(imag) == len(real):
                w = Zi.from_array(imag)
                re = z
                im = w
            else:
                raise Exception(f"Inputs incompatible: {real} and {imag}")
            super().__init__(re, im)

        # CLAUSE FA --------------------------------------------------------
        # Both real and imag are None

        elif real is None:
            re = 0
            if imag is None:
                im = 0
            else:
                raise Exception(f"If real is None, then imag must be None. But imag = {imag}")
            super().__init__(re, im)
        else:
            raise Exception(f"Unexpected combination of input types: {real} and {imag}")

    @classmethod
    def max_denominator(cls, denom=None):
        if denom is None:
            return cls.__MAX_DENOMINATOR
        elif isinstance(denom, int) and denom > 1:
            cls.__MAX_DENOMINATOR = denom
            return cls.__MAX_DENOMINATOR
        else:
            raise ValueError(f"Maximum denominator, {denom}, must be an integer > 1")

    @staticmethod
    def zero():
        return Qi(0, 0)
    
    @staticmethod
    def eye():
        return Qi(0, 1)
    
    @staticmethod
    def two():
        return Qi(1, 1)

    def norm(self):
        """Square of the usual norm."""
        return self.real ** 2 + self.imag ** 2

In [16]:
# Example Zi usage and testing:

if __name__ == "__main__":

    print("=== Qi Demo ===\n")
    
    print(f"{Zi() = }")
    print(f"{Zi(1) = }")
    print(f"{Zi.zero() = }")
    print(f"{Zi.eye() = }")
    print(f"{Zi.two() = }")
    print(f"{Zi(2.3, 3.8) = }")
    print(f"{Zi(-2.3, 3.8) = }")
    print(f"{Zi(2.3, -3.8) = }")
    print(f"{Zi(-2.3, -3.8) = }")
    print(f"{Zi(2.3, 4) = }")
    print(f"{Zi(-2.3, 4) = }")
    print(f"{Zi(2, 3.8) = }")
    print(f"{Zi(2, -3.8) = }")
    print(f"{Zi(2.3) = }")
    print(f"{Zi(2) = }")
    print(f"{Zi((2.3 - 3.7j)) = }")
    print(f"{Zi(-3.3j) = }")

    print("\n=== End of Demo ===\n")

=== Qi Demo ===

Zi() = Zi(0, 0)
Zi(1) = Zi(1, 0)
Zi.zero() = Zi(0, 0)
Zi.eye() = Zi(0, 1)
Zi.two() = Zi(1, 1)
Zi(2.3, 3.8) = Zi(2, 4)
Zi(-2.3, 3.8) = Zi(-2, 4)
Zi(2.3, -3.8) = Zi(2, -4)
Zi(-2.3, -3.8) = Zi(-2, -4)
Zi(2.3, 4) = Zi(2, 4)
Zi(-2.3, 4) = Zi(-2, 4)
Zi(2, 3.8) = Zi(2, 4)
Zi(2, -3.8) = Zi(2, -4)
Zi(2.3) = Zi(2, 0)
Zi(2) = Zi(2, 0)
Zi((2.3 - 3.7j)) = Zi(2, -4)
Zi(-3.3j) = Zi(0, -3)

=== End of Demo ===



In [18]:
# Example Qi usage and testing:

if __name__ == "__main__":

    print("=== Qi Demo ===\n")
    
    print(f"{Qi() = }")
    print(f"{Qi(1) = }")
    print(f"{Qi.zero() = }")
    print(f"{Qi.eye() = }")
    print(f"{Qi.two() = }")
    print(f"{Qi(2.5, 3.75) = }")
    print(f"{Qi(-2.25, 3.75) = }")
    print(f"{Qi(2.25, -3.75) = }")
    print(f"{Qi(-2.25, -3.75) = }")
    print(f"{Qi(2.25, 4) = }")
    print(f"{Qi(-2.25, 4) = }")
    print(f"{Qi(2, 3.75) = }")
    print(f"{Qi(2, -3.75) = }")
    print(f"{Qi(2.25) = }")
    print(f"{Qi(2) = }")
    print(f"{Qi((2.25 - 3.75j)) = }")
    print(f"{Qi(-3.25j) = }")

    print("\n=== End of Demo ===\n")

=== Qi Demo ===

Qi() = Qi(0, 0)
Qi(1) = Qi(1, 0)
Qi.zero() = Qi(0, 0)
Qi.eye() = Qi(0, 1)
Qi.two() = Qi(1, 1)
Qi(2.5, 3.75) = Qi(5/2, 15/4)
Qi(-2.25, 3.75) = Qi(-9/4, 15/4)
Qi(2.25, -3.75) = Qi(9/4, -15/4)
Qi(-2.25, -3.75) = Qi(-9/4, -15/4)
Qi(2.25, 4) = Qi(9/4, 4)
Qi(-2.25, 4) = Qi(-9/4, 4)
Qi(2, 3.75) = Qi(2, 15/4)
Qi(2, -3.75) = Qi(2, -15/4)
Qi(2.25) = Qi(9/4, 0)
Qi(2) = Qi(2, 0)
Qi((2.25 - 3.75j)) = Qi(9/4, -15/4)
Qi(-3.25j) = Qi(0, -13/4)

=== End of Demo ===

